# Usage

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import napari
import os
from MGFeatures import ER_length, img_from_tiles, label_cells, mask_it
from skimage import io
import gc

%load_ext autoreload
%autoreload 2

In [3]:
#single slice usage
folder = r'C:\Users\hanyi\Documents\MG\Cell_s1'
cells = img_from_tiles(folder, 1)

In [3]:
#multi slice usage - total number slices known
folder = r'C:\Users\hanyi\Documents\MG\ER'
#ER = [img_from_tiles(folder, i) for i in range(17)]
#ER = (img_from_tiles(folder, i) for i in range(17)) # if memory error
ER = img_from_tiles(folder, 1)

In [4]:
labels = label_cells(cells)

## ER length

In [14]:
#properties = {} #to be added to main function global variable
ER_length(ER, labels)

0.0
491.9066356420517
701.4873691797256
753.4284940958023
1217.5016479492188
1335.4844027757645
2730.6845359802246
2736.341390132904
3262.6988149881363
3274.598309636116
3280.2551637887955
3503.5516270399094
3505.5516270399094
3505.5516270399094
3516.380054116249
3751.8186523914337
3796.688709259033
4188.411578774452
4196.068432927132
190.12489008903503
246.75230658054352
258.40916073322296
312.4507907629013
977.3767576217651
1076.673220038414
1082.3300741910934
1132.8569855690002
1140.5138397216797
1146.1706938743591
1305.5676627159119
520.7422022819519
526.3990564346313
532.0559105873108
537.7127647399902
543.3696188926697
549.0264730453491
554.6833271980286
554.6833271980286
554.6833271980286
1021.8031650781631
60.72792184352875
422.53405153751373
428.1909056901932
441.2619733810425
607.9726501703262
607.9726501703262
685.9137746095657
763.5117536783218
804.8254618644714
970.5655761957169
983.050857424736
988.7077115774155
1126.9331068992615
1253.744289278984
1261.4011434316635
1312

## Area, ratio

In [5]:
folder = r'C:\Users\hanyi\Documents\MG\organelles_cyto'
organelles = img_from_tiles(folder, 1)

In [12]:
organelles_labels = mask_it(labels[20000:30000], organelles[20000:30000]) #subset of slice, memory error 

## trash

In [9]:
properties['ER_length']

KeyError: 'ER_length'

In [12]:
np.unique(labels)

StackViewNDArray([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,
                  14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
                  28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41,
                  42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55,
                  56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
                  70, 71, 72, 73, 74], dtype=uint8)

In [15]:
del ER
gc.collect()

947

In [26]:
import sys 
print(sys.getsizeof(organelles)/(1024**3))
print(sys.getsizeof(labels)/(1024**3))

3.5000001192092896
3.5000001415610313


## other useful functions

In [58]:
import sys 
sys.getsizeof(ER)/(1024**3)

3.5000001192092896

In [38]:
# free up space
del variable1, variable2
gc.collect()


854